In [1]:
%matplotlib inline
# Importing standard Qiskit libraries and configuring account
from qiskit import QuantumCircuit, execute, Aer, IBMQ
from qiskit.compiler import transpile, assemble
from qiskit.tools.jupyter import *
from qiskit.visualization import *
# Loading your IBM Q account(s)
provider = IBMQ.load_account()
provider = IBMQ.get_provider('ibm-q-internal', 'deployed', 'default')

/opt/conda/lib/python3.8/site-packages/qiskit/providers/ibmq/ibmqfactory.py:192: UserWarning: Timestamps in IBMQ backend properties, jobs, and job results are all now in local time instead of UTC.
  warnings.warn('Timestamps in IBMQ backend properties, jobs, and job results '


In [2]:
# initialization
import matplotlib.pyplot as plt
import numpy as np

# importing Qiskit
from qiskit import IBMQ, BasicAer
# from qiskit.providers.ibmq import least_busy
from qiskit.providers.ibmq import *
from qiskit import QuantumCircuit, ClassicalRegister, QuantumRegister, execute

# import basic plot tools
from qiskit.visualization import plot_histogram

In [3]:
n = 7 # number of qubits used to represent s

In [4]:
def wordToBV(s) :
    #convert text to binary
    a_byte_array = bytearray(s, "utf8")
    byte_list = []


    for byte in a_byte_array:
        binary_representation = bin(byte)
        byte_list.append(binary_representation[9-n:])
        #chop off the "0b" at the beginning. can also truncate the binary to fit on a device with N qubits
        #binary has 2 extra digits for "0b", so it starts at 9 for our 7 bit operation. 

    print(byte_list)
    
    circuit_array = []
    
    length = len(byte_list) 
    
    for i in range(length):
    
        s = byte_list[i]


        #do all  this stuff for every letter

        # We need a circuit with n qubits, plus one ancilla qubit
        # Also need n classical bits to write the output to
        bv_circuit = QuantumCircuit(n+1, n)

        # put ancilla in state |->
        bv_circuit.h(n)
        bv_circuit.z(n)

        # Apply Hadamard gates before querying the oracle
        for i in range(n):
            bv_circuit.h(i)

        # Apply barrier 
        bv_circuit.barrier()

        # Apply the inner-product oracle
        s = s[::-1] # reverse s to fit qiskit's qubit ordering
        for q in range(n):
            if s[q] == '0':
                bv_circuit.i(q)
            else:
                bv_circuit.cx(q, n)

        # Apply barrier 
        bv_circuit.barrier()

        #Apply Hadamard gates after querying the oracle
        for i in range(n):
            bv_circuit.h(i)

        # Measurement
        for i in range(n):
            bv_circuit.measure(i, i)
            
        circuit_array.append(bv_circuit)

    
    return circuit_array

In [5]:
circuit_to_run = wordToBV('A')

['1000001']


In [90]:
#run the first letter on a simulator
backend = provider.get_backend('ibmq_paris')

shots = 4096

# Execute the circuit on the backend
job = execute(circuit_to_run[0], backend, shots=shots, memory=True)

# monitor job
from qiskit.tools.monitor import job_monitor
job_monitor(job, interval = 2)

# Grab results from the job
result = job.result()

#get individual shots
memory = result.get_memory()

# need an array to drop all the results into
outputArray = []

#convert results to int and drop into array
for x in range(0, shots):
    outputArray.append(memory[x])
    
print(outputArray)

Job Status: job has successfully run
['0000011', '0110001', '1100011', '0111011', '1001010', '1000011', '1001011', '1001111', '0011101', '1000110', '1101011', '0100011', '1101011', '1101001', '1000011', '1001001', '0011001', '1000011', '0100011', '1111011', '0001101', '1100011', '1001011', '1000011', '1100001', '1000001', '1001010', '1011010', '1010011', '1101011', '1111111', '1000011', '0001010', '1100011', '1000011', '0000011', '1000010', '1011010', '0011000', '1000011', '0011011', '1000011', '1000011', '0001001', '1001011', '1100011', '1001001', '1000011', '1101000', '0100011', '1001000', '1001011', '1101001', '1000010', '0010001', '1101011', '1101011', '1001011', '1000011', '1000001', '1001010', '1000001', '1010011', '0010010', '1000111', '1000011', '1001011', '0001011', '1001010', '1111011', '1001110', '1001011', '0010011', '1100001', '1000010', '1001010', '0010011', '1000011', '1001010', '1000010', '1000011', '1000001', '0010001', '1101001', '1000011', '0000011', '1100011', '0000

In [91]:
%%capture cap --no-stderr
print(outputArray)

In [92]:
with open('output-typography-Z-paris.txt', 'w') as f:
    f.write(cap.stdout)